In [ ]:
# import pyjsonviewer

In [83]:
pyjsonviewer.view_data(json_data=resp)

NameError: name 'pyjsonviewer' is not defined

In [1]:
import requests
from pymongo import GEOSPHERE
import statistics
import json
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()

True

In [2]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost/companies")
db=client.get_database("companies")
c = db.get_collection("companies")


In [3]:
#use inputs if time
conditions = [
    ["design",5000,"m",5],
    ["school",8000,"m",4],
    ["Starbucks",1000,"s",7],
    ["airport",10000,"s",6],
    ["karaoke",2000,"m",5],
    ["vegan",5000,"s",7],
    ["basketball stadium",10000,"s",3],
    ["vet",6000,"s",8]
]

In [4]:
proj ={"name":1, "offices.country_code":1,"offices.latitude":1,
       "offices.longitude":1,"_id":0}
cond1 ={"tag_list":{"$regex": "games"}}
cond2 ={"total_money_raised": {"$regex":"\d.M"}}
seeds = list(c.find({"$and":[cond1,cond2]}, proj).sort("total_money_raised",-1))

In [9]:
len(seeds)

15

In [8]:
for n,seed in enumerate(seeds):
    for i,office in enumerate(seed["offices"]):
        if office["latitude"] == None:
            del seed["offices"][i:]
        if len(seed["offices"])==0:
            del seeds[n]

In [14]:
seeds

[{'name': 'Bigpoint',
  'offices': [{'country_code': 'DEU',
    'latitude': 53.5973055,
    'longitude': 9.9762519}]},
 {'name': 'eRepublik',
  'offices': [{'country_code': 'ESP',
    'latitude': 40.43094,
    'longitude': -3.6952896}]},
 {'name': 'Apaja',
  'offices': [{'country_code': 'FIN',
    'latitude': 60.171821,
    'longitude': 24.945167}]},
 {'name': 'Zynga',
  'offices': [{'country_code': 'USA',
    'latitude': 37.765158,
    'longitude': -122.404234}]},
 {'name': 'WildTangent',
  'offices': [{'country_code': 'USA',
    'latitude': 37.09024,
    'longitude': -95.712891}]},
 {'name': 'Heyzap',
  'offices': [{'country_code': 'USA',
    'latitude': 37.7905538,
    'longitude': -122.4041485}]},
 {'name': 'Metacafe',
  'offices': [{'country_code': 'USA',
    'latitude': 37.437328,
    'longitude': -122.159928}]},
 {'name': 'Zango',
  'offices': [{'country_code': 'USA',
    'latitude': 47.579208,
    'longitude': -122.155668}]},
 {'name': 'Hands-On Mobile',
  'offices': [{'country

In [12]:
locations = [type_point(office["longitude"],office["latitude"]) for seed in seeds for office in seed["offices"]]

In [13]:
locations[0]

{'type': 'Point', 'coordinates': [9.9762519, 53.5973055]}

In [11]:
def type_point(long,lat):
    return {"type": "Point", 
            "coordinates":[long,lat]}

In [14]:
client_id = os.getenv("Client_Id")
client_secret = os.getenv("Client_Secret")

#dondebuscamos = locations[n]

url_query = 'https://api.foursquare.com/v2/venues/explore'

In [15]:

map_name = ["venue", "name"]
map_lat = ["venue", "location", "lat"]
map_long = ["venue", "location", "lng"]
map_distance = ["venue", "location", "distance"]
map_direction = ["venue", "location", "formattedAddress"]

In [16]:
def getFromDict(dic, map_):
    from functools import reduce
    import operator
    return reduce(operator.getitem,map_,dic)

In [25]:
scoring=[]
for i,location in enumerate(locations):
    score = dict()
    for condition in conditions:
        parametros = {
            "client_id": client_id,
            "client_secret": client_secret,
            "v": "20180323",
            "ll": f"{location['coordinates'][1]}, {location['coordinates'][0]}",
            "query": f"{condition[0]}",
            "limit": 150}
        try:
            resp = requests.get(url_query, params=parametros).json()["response"]["groups"][0]["items"]
            newjson = []
            for dic in resp:
                forlist= {}
                forlist["name"]= getFromDict(dic,map_name)
                forlist["lat"]= getFromDict(dic,map_lat)
                forlist["long"]= getFromDict(dic,map_long)
                forlist["distance"]= getFromDict(dic,map_distance)
                forlist["direction"]= getFromDict(dic,map_direction)
                forlist["location"]= type_point(forlist["lat"],forlist["long"])
                newjson.append(forlist)
            
            with open(f'../data/jsons/{i}_{condition[0]}.json', 'w') as f:
                json.dump(newjson,f)
        
            basejson = db[f"{i}_{condition[0]}c"]
            ins = basejson.insert_many(newjson)
        except:
            basejson = db[f"{i}_{condition[0]}c"]
            print("resp fail")

       
        #basejson.createIndex( { location : "2dsphere" } )
        try:
            basejson.create_index([("location", GEOSPHERE)])
            
            query = {"location": {"$near":{"$geometry": location, "$maxDistance": condition[1]}}}
        
            l= list(basejson.find(query))
        except:
            l=[]
            print(" fail")

        distances = [item["distance"] for item in l]
        if len(l) != 0:
            if condition[2]=="s":
                 score[condition[0]]= condition[3]*1/min(distances)
            else:
                 score[condition[0]]= condition[3]*1/statistics.mean(distances)
        else:
            score[condition[0]]= 0
    score["total"]= sum(list(score.values()))
    score["location"]=location
    scoring.append(score)


query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
resp fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
resp fail
query fail
query fail
resp fail
query fail
resp fail
query fail
resp fail
query fail
resp fail
query fail
resp fail
query fail
resp fail
query fail
resp fail
query fail
resp fail
query fail
resp fail
query fail
resp fail
query fail
resp fail
query fail
resp fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
query fail
qu

In [27]:
basejson.find_one()

{'_id': ObjectId('6136a6271df64a26c60c5186'),
 'name': 'LIV',
 'lat': 53.579121911537506,
 'long': 9.956514912770784,
 'distance': 2407,
 'direction': ['Lutterothstraße 8', '20255 Hamburg', 'Deutschland'],
 'location': {'type': 'Point', 'coordinates': [9.9762519, 53.5973055]}}

In [23]:
sc = pd.DataFrame(scoring)
sc

,design,school,Starbucks,airport,karaoke,vegan,basketball stadium,vet,total,location
0,0.001587,0,0,0,0,0,0,0,0.001587,"{'type': 'Point', 'coordinates': [9.9762519, 5..."
1,0.000000,0,0,0,0,0,0,0,0.000000,"{'type': 'Point', 'coordinates': [-3.6952896, ..."
2,0.000000,0,0,0,0,0,0,0,0.000000,"{'type': 'Point', 'coordinates': [24.945167, 6..."
3,0.000000,0,0,0,0,0,0,0,0.000000,"{'type': 'Point', 'coordinates': [-122.404234,..."
4,0.000000,0,0,0,0,0,0,0,0.000000,"{'type': 'Point', 'coordinates': [-95.712891, ..."
5,0.000000,0,0,0,0,0,0,0,0.000000,"{'type': 'Point', 'coordinates': [-122.4041485..."
6,0.000000,0,0,0,0,0,0,0,0.000000,"{'type': 'Point', 'coordinates': [-122.159928,..."
7,0.000000,0,0,0,0,0,0,0,0.000000,"{'type': 'Point', 'coordinates': [-122.155668,..."
8,0.000000,0,0,0,0,0,0,0,0.000000,"{'type': 'Point', 'coordinates': [-122.419204,..."
9,0.000000,0,0,0,0,0,0,0,0.000000,"{'type': 'Point', 'coordinates': [-122.335943,..."


In [130]:
parametros = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{locations[0]['coordinates'][1]}, {locations[0]['coordinates'][0]}",
    "query": "school",
    "limit": 150   
}
resp = requests.get(url_query, params=parametros).json()["response"]["groups"][0]["items"]

In [131]:
resp[0]["venue"]

{'id': '4f1fa8cbe4b01a2ba1a72a76',
 'name': 'Grundschule Loewenstrasse',
 'contact': {},
 'location': {'address': 'Loewenstrasse 58',
  'lat': 53.58634733157745,
  'lng': 9.973167436096483,
  'labeledLatLngs': [{'label': 'display',
    'lat': 53.58634733157745,
    'lng': 9.973167436096483}],
  'distance': 1236,
  'postalCode': '20251',
  'cc': 'DE',
  'city': 'Hamburg',
  'state': 'Hamburg',
  'country': 'Deutschland',
  'formattedAddress': ['Loewenstrasse 58', '20251 Hamburg', 'Deutschland']},
 'categories': [{'id': '4bf58dd8d48988d13b941735',
   'name': 'School',
   'pluralName': 'Schools',
   'shortName': 'School',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 0,
  'usersCount': 0,
  'checkinsCount': 0,
  'visitsCount': 0},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
  'marked': False,
  'unconfirmedCount': 0},
 'photos': {'count': 0, 'groups': []},


In [ ]:

import geopandas as gpd
gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.longitud, df.latitud))
gdf.sample()